# * Sales Channel

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

In [2]:
''' Input parameter '''

op_dir = 'data'
op_sales_channel_file = f'sales_channel_{str_curr_dt}'

# v_year = 0
# v_month_start = 0
# v_month_end = 0
# v_date = 20250101

print(f'\nParameter input...\n')
print(f'   -> op_dir: {op_dir}')
print(f'   -> op_sales_channel_file: {op_sales_channel_file}')
# print(f'\n   -> v_year: {v_year}')
# print(f'   -> v_month_start: {v_month_start}')
# print(f'   -> v_month_end: {v_month_end}')
# print(f'\n   -> v_date: {v_date}')


Parameter input...

   -> op_dir: data
   -> op_sales_channel_file: sales_channel_20251208


## Import Transaction
-> DASH_PERFORMANCE

In [3]:
''' Execute transaction '''


# Input parameter
v_start_date = 20250101
print(f'\nParameter input...')
print(f'   -> v_start_date: {v_start_date}')

curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
print(f'\nData as of {curr_datetime}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()

query = (f"""
    SELECT /*+PARALLEL(8)*/ 
        TM_KEY_MTH, COMPANY, PRODUCT, SUB_PRODUCT, GP_SKU, SPECIAL_SKU, PROJECT_GROUP, CUST_TYPE, BU_CHANNEL, TDS_CHANNEL, TDS_SUB_CHANNEL, GROUP_TDS_SPECIAL, TDS_SPECIAL_CHANNEL, CHANNEL_GROUP, BUNDLE_TYPE
        , ORGID_H, HOP_HINT, D_CLUSTER, ORGID_G, TDS_GMD, TDS_SGMD, DISTRICT_TH, DISTRICT_EN
        , A.CCAA, O.PROVINCE_CD, O.PROVINCE_ENG, O.PROVINCE_TH
        , TARGET, ACTUAL, RUNRATE, TARGET_VALUE, ACTIVATION_VALUE, ACTIVATION_VALUE_RUNRATE
        
    FROM (
        SELECT SUBSTR(TM_KEY_DAY, 1, 6) TM_KEY_MTH
            , COMPANY, PRODUCT, SUB_PRODUCT, GP_SKU, SPECIAL_SKU, PROJECT_GROUP, CUST_TYPE, BU_CHANNEL, TDS_CHANNEL, TDS_SUB_CHANNEL, GROUP_TDS_SPECIAL, TDS_SPECIAL_CHANNEL, CHANNEL_GROUP, BUNDLE_TYPE 
            , ORGID_H, HOP_HINT, D_CLUSTER, ORGID_G, TDS_GMD, TDS_SGMD, DISTRICT_TH, DISTRICT_EN
            , SUBSTR(PARTNER_CCAATT, 1, 4) AS CCAA --, PARTNER_CCAATT, PARTNER_SCAB
            , SUM(TARGET) TARGET, SUM(ACTUAL) ACTUAL, SUM(RUNRATE) RUNRATE, SUM(TARGET_VALUE) TARGET_VALUE, SUM(ACTIVATION_VALUE) ACTIVATION_VALUE, SUM(ACTIVATION_VALUE_RUNRATE) ACTIVATION_VALUE_RUNRATE
            
        FROM CDSSHRE.DASH_PERFORMANCE
        WHERE TM_KEY_DAY >= {v_start_date}
        AND ( (SUB_PRODUCT = 'POSTPAY' AND CUST_TYPE = 'INDY')
            OR (SUB_PRODUCT IN ('PREPAY', 'INFLOW M1', 'INFLOW M2', 'AP', 'APD1') AND CUST_TYPE = 'INDY')
            OR SUB_PRODUCT = 'TOL' --AND GP_SKU = 'Consumer Service'
            OR SUB_PRODUCT IN ('TVS NOW APP', 'CMDU') )

        GROUP BY SUBSTR(TM_KEY_DAY, 1, 6) 
            , COMPANY, PRODUCT, SUB_PRODUCT, GP_SKU, SPECIAL_SKU, PROJECT_GROUP, CUST_TYPE, BU_CHANNEL, TDS_CHANNEL, TDS_SUB_CHANNEL, GROUP_TDS_SPECIAL, TDS_SPECIAL_CHANNEL, CHANNEL_GROUP, BUNDLE_TYPE 
            , ORGID_H, HOP_HINT, D_CLUSTER, ORGID_G, TDS_GMD, TDS_SGMD, DISTRICT_TH, DISTRICT_EN
            , SUBSTR(PARTNER_CCAATT, 1, 4)
    ) A
    LEFT JOIN (
        SELECT DISTINCT SUBSTR(CCAATT,1,4) AS CCAA--, DISTRICT_EN
            , SUBSTR(CCAATT,1,2) AS PROVINCE_CD, PROVINCE_ENG, PROVINCE_TH
        FROM CDSAPPO.DIM_MOOC_AREA
        WHERE TEAM_CODE <> 'ไม่ระบุ' AND REMARK <> 'Dummy'
    ) O
        ON O.CCAA = A.CCAA
""")


try:
    # Create Dataframe
    src_cur.execute(query)
    rows = src_cur.fetchall()
    src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\nDataFrame: {src_df.shape[0]} rows, {src_df.shape[1]} columns')

    # Generate CSV file
    src_df.to_csv(f'{op_dir}/{op_sales_channel_file}.csv', index=False, encoding='utf-8')
    print(f'\n   -> Generate "{op_sales_channel_file}.csv" successfully')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...
   -> v_start_date: 20250101

Data as of 2025-12-08, 09:33:57

DataFrame: 1577422 rows, 33 columns

   -> Generate "sales_channel_20251208.csv" successfully


## Review

In [4]:
src_df

,TM_KEY_MTH,COMPANY,PRODUCT,SUB_PRODUCT,GP_SKU,SPECIAL_SKU,PROJECT_GROUP,CUST_TYPE,BU_CHANNEL,TDS_CHANNEL,...,CCAA,PROVINCE_CD,PROVINCE_ENG,PROVINCE_TH,TARGET,ACTUAL,RUNRATE,TARGET_VALUE,ACTIVATION_VALUE,ACTIVATION_VALUE_RUNRATE
0,202502,DTAC,PREPAY,APD1,TOURIST,TOURIST,None,INDY,DTAC Channel,Unidentified,...,1903,19,Saraburi,สระบุรี,0.0,143,143.0,0.00,0.00,0.00
1,202502,TRUE,PREPAY,APD1,MASS,MASS,MASS,INDY,Non-Shop,Lotus Non Shop,...,1010,10,Bangkok,กรุงเทพมหานคร,0.0,7,7.0,0.00,0.00,0.00
2,202508,TRUE,PREPAY,INFLOW M1,MASS,MASS,MASS,INDY,7-11,7-11,...,2508,25,Prachin Buri,ปราจีนบุรี,0.0,0,0.0,0.00,59292.76,59292.76
3,202511,TRUE,PREPAY,INFLOW M2,MIGRANT,MASS,MASS,INDY,Non-Shop,Wholesales,...,7401,74,Samut Sakhon,สมุทรสาคร,0.0,0,0.0,0.00,211465.40,211465.40
4,202507,DTAC,PREPAY,INFLOW M1,MASS,MASS,None,INDY,DTAC Channel,Unidentified,...,9201,92,Trang,ตรัง,0.0,0,0.0,0.00,26761.15,26761.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1577417,202506,TRUE,PREPAY,INFLOW M2,MASS,MASS,MASS,INDY,Non-Shop,Lotus Non Shop,...,3007,30,Nakhon Ratchasima,นครราชสีมา,0.0,0,0.0,0.00,0.00,0.00
1577418,202506,TRUE,POSTPAY,POSTPAY,Convergence,Convergence,A0-MASS,INDY,Non-Shop,True Shop Double7,...,1104,11,Samut Prakan,สมุทรปราการ,0.0,1,1.0,0.00,64.00,64.00
1577419,202506,TRUE,POSTPAY,POSTPAY,SIM Only,SIM Only,A0-MASS,INDY,Non-Shop,Dealer,...,1106,11,Samut Prakan,สมุทรปราการ,0.0,2,2.0,0.00,2498.00,2498.00
1577420,202506,TRUE,PREPAY,INFLOW M1,TOURIST,TOURIST,TOURIST,INDY,7-11,7-11,...,4907,49,Mukdahan,มุกดาหาร,0.0,0,0.0,5.59,0.00,0.00
